## Currency Conversion Tool 

In [48]:
from langchain_community.tools import tool
from langchain_core.tools import InjectedToolArg
from langchain_core.messages import HumanMessage
from langchain_groq import ChatGroq
from dotenv import load_dotenv
import os
import requests
from typing import Annotated
import json
load_dotenv()

exchange_rate_api = os.getenv('EXCHANGE_RATE_API')

In [49]:
# tool creation

@tool 
def get_conversion_factor(base_currency: str, target_currency: str) -> float:
    """This function fetches the conversion factor between base_currency and target_currency"""
    url = f'https://v6.exchangerate-api.com/v6/{exchange_rate_api}/pair/{base_currency}/{target_currency}'
    response = requests.get(url)
    return response.json()

@tool
def convert(base_currency_value: int, conversion_rate: Annotated[float, InjectedToolArg] ) -> float:
    """given a currency conversion rate this function calculates the target currency value from a given base currency value"""
    
    return base_currency_value * conversion_rate


In [50]:
# tool biniding
# Creating a model
llm = ChatGroq(model='meta-llama/llama-4-maverick-17b-128e-instruct')

llm_with_tools = llm.bind_tools([get_conversion_factor, convert])

In [51]:
messages = [HumanMessage("Can you give me conversion factor of USD to PKR and also convert 50 USD to PKR")]
messages

[HumanMessage(content='Can you give me conversion factor of USD to PKR and also convert 50 USD to PKR', additional_kwargs={}, response_metadata={})]

In [52]:
ai_message = llm_with_tools.invoke(messages)
messages.append(ai_message)
ai_message.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'base_currency': 'USD', 'target_currency': 'PKR'},
  'id': 'e0yqnmj30',
  'type': 'tool_call'},
 {'name': 'convert',
  'args': {'base_currency_value': 50},
  'id': 'rxs36wm45',
  'type': 'tool_call'}]

In [53]:

for tool_call in ai_message.tool_calls:
  # execute the 1st tool and get the value of conversion rate
  if tool_call['name'] == 'get_conversion_factor':
    tool_message1 = get_conversion_factor.invoke(tool_call)
    # fetch this conversion rate
    conversion_rate = json.loads(tool_message1.content)['conversion_rate']
    # append this tool message to messages list
    messages.append(tool_message1)
  # execute the 2nd tool using the conversion rate from tool 1
  if tool_call['name'] == 'convert':
    # fetch the current arg
    tool_call['args']['conversion_rate'] = conversion_rate
    tool_message2 = convert.invoke(tool_call)
    messages.append(tool_message2)



In [54]:
messages

[HumanMessage(content='Can you give me conversion factor of USD to PKR and also convert 50 USD to PKR', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'e0yqnmj30', 'function': {'arguments': '{"base_currency":"USD","target_currency":"PKR"}', 'name': 'get_conversion_factor'}, 'type': 'function'}, {'id': 'rxs36wm45', 'function': {'arguments': '{"base_currency_value":50}', 'name': 'convert'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 54, 'prompt_tokens': 754, 'total_tokens': 808, 'completion_time': 0.092235206, 'prompt_time': 0.021082919, 'queue_time': 0.035089611, 'total_time': 0.113318125}, 'model_name': 'meta-llama/llama-4-maverick-17b-128e-instruct', 'system_fingerprint': 'fp_c527aa4474', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--2c013dd0-01f1-4526-aef9-19990d984225-0', tool_calls=[{'name': 'get_conversion_factor', 'args': {'base_currency': 'U

In [55]:
result = llm_with_tools.invoke(messages)
result.content

'The conversion factor of USD to PKR is 283.608. \n50 USD is equivalent to 14180.4 PKR.'